In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
data = pd.read_excel('../dataSet/dis.xlsx', header= None)
data = np.array(data)

In [3]:
class HNN():
    def __init__(self, A, B, C, D, u0, delta, max_epochs, data):
        self.A  = A  
        self.B  = B
        self.C  = C
        self.D  = D
        self.u0 = u0 #状态方程参数u0
        self.data  = data  # 城市距离数据
        self.node  = len(data)  # 总节点数 = self.node * self.node
        self.delta = delta  # 状态方程步长参数delta
        self.max_epochs = max_epochs #最大迭代次数
        self.U = np.random.uniform(low = -0.1, high = 0.1, size = (self.node, self.node)) #U矩阵
        self.V = np.zeros((self.node, self.node))  #神经元矩阵
        self.E = float("inf")  # 当前能量值
        
    def getEnergy(self):
        # 计算当前能量
        # t1 t2 t3为解的约束项  t4为最优解相关项
        t1 = t2 = t3 = t4 = 0
        for x in range(self.node):
            for i in range(self.node):
                for j in range(self.node):
                    if i != j:
                        t1 += self.V[x, i] * self.V[x, j]
        for y in range(self.node):
            for i in range(self.node):
                for j in range(self.node):
                    if i != j:
                        t2 += self.V[i, y] * self.V[j, y]
        t3 = np.power((np.sum(self.V) - self.node), 2)
        for x in range(self.node):
            for y in range(self.node):
                if y != x:
                    for i in range(self.node):
                        t4 += self.data[x, y] * self.V[x, i] * (self.V[y, (i + 1) % self.node] + self.V[y, (i - 1) % self.node])
        energy = 0.5 * (self.A * t1 + self.B * t2 + self.C * t3 + self.D * t4)
        #print('energy',energy)
        return energy
    
    def getDeltaU(self):
        # 计算△U
        # t1 t2 t3为解的约束项  t4为最优解相关项
        deltaU = np.zeros((self.node, self.node))
        for x in range(self.node):
            for i in range(self.node):
                t1 = sum(self.V[x]) - self.V[x, i]
                t2 = sum(self.V.T[i]) - self.V[x, i]
                t3 = np.sum(self.V) - self.node
                t4 = 0
                for y in range(self.node):
                    if y != x :
                        t4 += self.data[x, y]*(self.V[y, (i + 1) % self.node] + self.V[y, (i - 1) % self.node])
                deltaU[x,i] = -(self.A * t1 + self.B * t2 + self.C * t3 + self.D * t4)
        #print('deltaU',deltaU)
        return deltaU
    
    def updateU(self, deltaU):
        # 更新U矩阵 
        # U += △U * δ 
        self.U += deltaU * self.delta
        #print('U', self.U)
        return
    
    def updateV(self):
        # 更新V矩阵
        # Vxi = 0.5 * (1+tanh(Uxi/u0)) 
        for x in range(self.node):
            for i in range(self.node):
                self.V[x, i] = 0.5 * (1 + np.tanh(self.U[x, i] / self.u0))
        #print('V', self.V)
        return

    def getdist(self):
        # 计算当前解
        dist = 0
        route = np.argmax(self.V, axis = 0) 
        if len(set(route)) != len(route): # 路径有重复
            return np.nan
        for i in range(self.node):
            dist += self.data[route[i], route[(i + 1) % self.node]]
        return dist
    
    def run(self):   
        epoch = 0 #当前迭代次数
        while epoch < max_epochs: #小于最大迭代次数
            epoch = epoch + 1
            deltaU = self.getDeltaU() #计算△U
            self.updateU(deltaU)  #更新U矩阵
            self.updateV() #更新V矩阵
            energy = self.getEnergy() #计算能量值
            if self.E - energy > 1e-10:  #是否收敛
                self.E = energy
            else:
                break
        dist = self.getdist()  #当前解
        if np.isnan(dist):
            epoch = np.nan
        return epoch, dist
      

In [4]:
A  = 500
B  = 500
C  = 500
D  = 200
u0 = 0.02
delta = 0.00001 
max_epochs = 1000

In [ ]:
epochs  = list() #100次实验迭代次数列表
mindist = list() #100次实验迭最终解列表
start = time.process_time()  #计时开始
for i in range(100): #重复100次实验
    hnn = HNN(A, B, C, D, u0, delta, max_epochs, data)
    epoch, dist = hnn.run()
    epochs.append(epoch)
    mindist.append(dist)
end = time.process_time()   #计时结束
print(epochs)
print(mindist)
#对一百次实验取平均值
print('平均迭代次数:', round(np.nanmean(epochs), 2))
print('平均路径长度:', round(np.nanmean(mindist), 2))
print('运行总时间: %s (s)'%(end-start))